In [1]:
import matplotlib.pyplot as plt
import cloudpickle
import numpy as np

path = "output/flex/results.pkl"

with open(path, "rb") as file:
    results = cloudpickle.load(file)

results = list(results)
for rollout in results:
    for i in range(1,15):
        aid = f"H{i}"
        agent_actions = []
        agent_demand = []
        agent_charge = []
        agent_supply = []
        agent_prod = []
        agent_actions += list(rollout.actions_for_agent(aid))
        agent_demand += list(rollout.metrics[f"{aid}/current_load"])
        agent_charge += list(rollout.metrics[f"{aid}/current_charge"])
        agent_supply += list(rollout.metrics[f"{aid}/current_supply"])
        agent_prod += list(rollout.metrics[f"{aid}/current_prod"])

        for idx, action in enumerate(agent_actions):
            if action is None:
                agent_actions[idx] = agent_actions[idx + 1] if idx + 1 < len(agent_actions) else 6

        start = 48*30*8 - 48*2
        end = 48*30*8 - 48*1

        agent_actions = agent_actions[start:end]
        agent_demand = agent_demand[start:end]
        agent_charge = agent_charge[start:end]
        agent_supply = agent_supply[start:end]
        agent_prod = agent_prod[start:end]


        fig, ax1 = plt.subplots()

        ax2 = ax1.twinx()
        ax2.scatter(range(len(agent_actions)), agent_actions, label="Action", color='tab:green', alpha=0.5)
        ax2.set_ylabel('Action')
        ax2.set_yticks([0, 1, 2, 3, 4, 5])
        ax2.set_yticklabels(['buy', 'buycharge', 'sell', 'sellcharge', 'charge', 'noop'])
        ax2.legend(loc='upper right')
        
        ax1.plot(agent_demand, label="Demand", color='tab:blue')
        ax1.plot(agent_charge, label="Charge", color='tab:orange')
        ax1.plot(agent_supply, label="Supply", color='tab:red')
        ax1.plot(agent_prod, label="Production", color='tab:purple')
        ax1.set_xlabel('Time step')
        ax1.set_ylabel('Demand/Charge')
        ax1.legend(loc='upper left')


        fig.tight_layout()
        plt.savefig(f"demand_charge_action_{aid}.png")
        plt.close()




        # Plot distribution of agent action per step for all rollouts
        # plt.hist(agent_actions, bins=np.arange(-0.5, 6, 1), edgecolor='black')
        # plt.xticks(np.arange(6))
        # plt.title("Distribution of action values")
        # plt.xlabel("Agent action")
        # plt.ylabel("Frequency")
        # plt.savefig(f"action_dist_{aid}.png")
        # plt.close()


/Users/antonlenander/anaconda3/envs/phantom/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-08 13:09:02,657	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-08 13:09:02,862	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-08 13:09:03,300	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


0    1.25372
1    1.23378
2    1.20766
3    1.19457
4    1.17574
Name: SpotPriceDKK, dtype: float64
